# Momentum Decile Performance

The [Fama-French](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_10_port_form_pr_12_2_daily.html) data-set has returns for portfolios constructed out of each decile of prior returns. Labeled LO_PRIOR, PRIOR_2..9 and HI_PRIOR, they represent portfolios who's prior returns were the lowest through to the highest.

Here, we plot the returns of these portfolios to get a sense for how they have behaved through time.

The documentation for the Fama-French data-set can be found [here](https://plutopy.readthedocs.io/en/latest/FamaFrench.html) and [here](https://shyams80.github.io/plutoR/docs/reference/FamaFrench-class.html)

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(reshape2)
library(PerformanceAnalytics)
library(ggrepel)
library(tbl2xts)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")

#initialize
famaFrench <- FamaFrench()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:b

In [2]:
#startDt <- (famaFrench$MomentumDaily() %>% summarize(MAX = min(TIME_STAMP)) %>% collect())$MAX[[1]]
startDt <- as.Date("2015-01-01")

valueWtd <- famaFrench$MomentumDaily() %>%
    filter(RET_TYPE == 'AVWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has decile names HI_PRIOR...LO_PRIOR
    # we want HI_PRIOR...LO_PRIOR as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group)
    
valueWtdPx <- valueWtd %>% 
    # transform the return stream to prices
    # and convert to xts
    mutate_each(list(~ cumprod(. + 1) ), -DATE) %>% 
    tbl_xts() 

print(head(valueWtdPx))

valueWtdRets <- apply(valueWtdPx, 2, yearlyReturn)
print(valueWtdRets)

valueWtdRet <- valueWtd %>% 
    tbl_xts()

            HI_PRIOR  LO_PRIOR   PRIOR_2   PRIOR_3   PRIOR_4   PRIOR_5
2015-01-02 1.0004000 1.0028000 0.9981000 0.9989000 0.9986000 0.9985000
2015-01-05 0.9854940 0.9740196 0.9645638 0.9757255 0.9733354 0.9777312
2015-01-06 0.9759347 0.9590197 0.9475875 0.9602115 0.9598061 0.9646296
2015-01-07 0.9923305 0.9595951 0.9547892 0.9670290 0.9645091 0.9733113
2015-01-08 1.0153525 0.9813780 0.9720709 0.9852091 0.9799413 0.9903442
2015-01-09 1.0095650 0.9766673 0.9661412 0.9787068 0.9696519 0.9800446
             PRIOR_6   PRIOR_7   PRIOR_8   PRIOR_9
2015-01-02 0.9995000 0.9973000 0.9989000 0.9973000
2015-01-05 0.9785105 0.9804456 0.9828177 0.9840359
2015-01-06 0.9662791 0.9738766 0.9728913 0.9798046
2015-01-07 0.9760385 0.9852710 0.9876792 0.9945996
2015-01-08 0.9929240 1.0009368 1.0045685 1.0098170
2015-01-09 0.9829948 0.9905271 0.9938196 1.0034551


ERROR: Error in array(ans, c(len.a%/%d2, d.ans), if (!is.null(names(dn.ans)) || : length of 'dimnames' [1] not equal to array extent


In [3]:
plotAnnualReturns <- function(yearlies, mainTitle){
    yDf <- data.frame(yearlies)
    yDf$T <- year(index(yearlies))

    toPlot <- melt(yDf, id='T')

    ggplot(toPlot, aes(x=T, y=value, fill=variable)) +
        theme_economist() +
        geom_bar(stat="identity", position=position_dodge()) +
        scale_x_continuous(labels=yDf$T, breaks=yDf$T) +
        geom_text_repel(aes(label= round(value, 2)), position = position_dodge(0.9)) +
        coord_flip() +
        labs(x='', y='(%)', fill='', title=mainTitle, subtitle="Annual Returns") +
        annotate("text", x=max(yDf$T), y=min(toPlot$value), 
                 label = "@StockViz", hjust=1.1, vjust=-1.1, 
                 col="white", cex=6, fontface = "bold", alpha = 0.8)  
}

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)